In [1]:
import pickle as pk
from tqdm import tqdm
import numpy as np
index_map = {'医学': 0,
 '工程技术': 1,
 '生物学': 2,
 '化学': 3,
 '物理与天体物理': 4,
 '农林科学': 5,
 '材料科学': 6,
 '计算机科学': 7,
 '环境科学与生态学': 8,
 '地球科学': 9,
 '心理学': 10,
 '数学': 11,
 '法学': 12,
 '经济学': 13,
 '管理学': 14,
 '人文科学': 15,
 '教育学': 16,
 '综合性期刊':17}

In [2]:
# 跨学科程度
from math import isnan
paper2idDegree = pk.load(open('../../DataCrossBoundaryPerspective_InterdisciplinaryResearch/paper2idDegree.pk', 'rb'))
for k,v in tqdm(paper2idDegree.items()):
    if isnan(v):
        paper2idDegree[k] = 0

100%|██████████| 33613683/33613683 [00:06<00:00, 5472590.55it/s]


In [3]:
# 参考文献数量
referenceCounter = pk.load(open('../../DataCrossBoundaryPerspective_InterdisciplinaryResearch/referenceCounter.pk','rb'))

In [4]:
#年份
paperYear = pk.load(open('../../DataCrossBoundaryPerspective_InterdisciplinaryResearch/paperYear.pk', 'rb'))

In [5]:
#团队规模
paperAuthorNum = pk.load(open('../../DataCrossBoundaryPerspective_InterdisciplinaryResearch/paperAuthorNum.pk', 'rb'))

In [6]:
# 期刊学科对应
magid2bigcategory = pk.load(open('../../DataCrossBoundaryPerspective_InterdisciplinaryResearch/magid2bigcategory.pk', 'rb'))

In [7]:
# 这里是我们考虑的文献名录，约3000万篇。我们只考虑这3000多万篇文献中的引用关系。
paperSet = pk.load(open('../../DataCrossBoundaryPerspective_InterdisciplinaryResearch/paperJournalsetCas.pk', 'rb'))

In [8]:
# 文献期刊对应
paper2journalid = pk.load(open('../../DataCrossBoundaryPerspective_InterdisciplinaryResearch/paper2journalid.pk', 'rb'))

In [9]:
# 计算每年、每个学科的平均跨学科程度
year_field_iddegree_list = {}
for paper, iddegree in tqdm(paper2idDegree.items()):
    try:
        if paperYear[paper] not in year_field_iddegree_list:
            year_field_iddegree_list[paperYear[paper]] = {index_map[magid2bigcategory[paper2journalid[paper]]]:[iddegree]}
        else:
            if index_map[magid2bigcategory[paper2journalid[paper]]] not in year_field_iddegree_list[paperYear[paper]]:
                year_field_iddegree_list[paperYear[paper]][index_map[magid2bigcategory[paper2journalid[paper]]]] = [iddegree]
            else:
                year_field_iddegree_list[paperYear[paper]][index_map[magid2bigcategory[paper2journalid[paper]]]].append(iddegree)
    except:
        pass

100%|██████████| 33613683/33613683 [01:28<00:00, 379077.56it/s]


In [10]:
for year,field_list in tqdm(year_field_iddegree_list.items()):
    for field, iddegreeList in field_list.items():
        year_field_iddegree_list[year][field] = np.array(iddegreeList)

100%|██████████| 163/163 [00:06<00:00, 24.54it/s]


In [11]:
# 计算平均值
# year_field_avgiddegree = {}
# for year,field_list in tqdm(year_field_iddegree_list.items()):
#     year_field_avgiddegree[year] = {}
#     for field, iddegreeList in field_list.items():
#         year_field_avgiddegree[year][field] = sum(iddegreeList)/len(iddegreeList)

In [12]:
# 计算世界各年的平均值
# year_iddegree_list = {}
# for paper, iddegree in tqdm(paper2idDegree.items()):
#     try:
#         if paperYear[paper] not in year_iddegree_list:
#             year_iddegree_list[paperYear[paper]] = [iddegree]
#         else:
#             year_iddegree_list[paperYear[paper]].append(iddegree)
#     except:
#         continue

In [13]:
# year_avg_iddegree = {}
# for year, iddegree_list in year_iddegree_list.items():
#     year_avg_iddegree[year] = sum(iddegree_list)/len(iddegree_list)
# year_avg_iddegree

In [14]:
# pk.dump(year_avg_iddegree,open('../../DataCrossBoundaryPerspective_InterdisciplinaryResearch/year_avg_iddegree.pk', 'wb'))

In [15]:
# pk.dump(year_field_avgiddegree,open('../../DataCrossBoundaryPerspective_InterdisciplinaryResearch/year_field_avgiddegree.pk', 'wb'))

下面研究管理学领域的情况

In [16]:
# 主要负责文献：开始年份
def StartYearpublist(publist, Typestr='both'):
    # yearPublist  {2023:{'rank1':set(), 'rankLast':set(), 'others':set() }, ...}
    if Typestr == 'both':
        for year, yearpublist in publist.items():
            if len(yearpublist['rank1']) > 0 or len(yearpublist['rankLast']) > 0:
                return int(year)
    elif Typestr == 'rank1':
        for year, yearpublist in publist.items():
            if len(yearpublist['rank1']) > 0:
                return int(year)
    elif Typestr == 'rankLast':
        for year, yearpublist in publist.items():
            if len(yearpublist['rankLast']) > 0:
                return int(year)
    return -1
# 主要负责文献：休止年份
def EndYearpublist(publist, Typestr='both'):
    if Typestr == 'both':
        for year, yearpublist in sorted(publist.items(), reverse=True):
            if len(yearpublist['rank1']) > 0 or len(yearpublist['rankLast']) > 0:
                return int(year)
    elif Typestr == 'rank1':
        for year, yearpublist in sorted(publist.items(), reverse=True):
            if len(yearpublist['rank1']) > 0:
                return int(year)
    elif Typestr == 'rankLast':
        for year, yearpublist in sorted(publist.items(), reverse=True):
            if len(yearpublist['rankLast']) > 0:
                return int(year)
    return -1
# 主要负责文献：年份跨度
def SpanYearpublist(publist, Typestr='both'):
    return EndYearpublist(publist,Typestr) - StartYearpublist(publist,Typestr)

# 主要负责文献：真实年份跨度
def TrueSpanYearpublist(publist, Typestr='both'):
    # yearPublist  {2023:{'rank1':set(), 'rankLast':set(), 'others':set() }, ...}
    truthSpan = 0
    if Typestr == 'both':
        for _, yearpublist in publist.items():
            if len(yearpublist['rank1']) > 0 or len(yearpublist['rankLast']) > 0:
                truthSpan += 1
    elif Typestr == 'rank1':
        for _, yearpublist in publist.items():
            if len(yearpublist['rank1']) > 0:
                truthSpan += 1
    elif Typestr == 'rankLast':
        for _, yearpublist in publist.items():
            if len(yearpublist['rankLast']) > 0:
                truthSpan += 1
    return truthSpan

In [17]:
# 主要参与者CAS学科BreakThrough的时间点(采用中科院分区), 参数cutYear是纳入观察的年数；修改该函数，只关心both（关键作者）、指定学科；输出跳转作品及以前的全部关键文献，没有跳转过就输出0
def BreakthroughForCAS_IDRidentify(publist, cutYear=100, initialSpan=1, TypeStr='both',Targetfield='管理学'):
    # yearPublist  {2023:{'rank1':set(), 'rankLast':set(), 'others':set() }, ...}
    # cutYear是控制的年数，必须比initialSpan大1或以上
    if cutYear <= initialSpan:
        raise('初始学科年份与生涯长度不匹配')
    initialSet = set()
    yearid = 0
    corepaperlist = []
    if TypeStr == 'both':
        for _, yearpublist in publist.items():
            # 是第一作者或末位作者
            if len(yearpublist['rank1']) > 0 or len(yearpublist['rankLast']) > 0:
                yearid += 1
                # 如果已经到了cutYear了，直接结束循环（到此，强行结束该研究者的职业生涯）。
                if yearid > cutYear:
                    break
                # 处女作之年，要初始化
                if len(initialSet) == 0 or yearid <= initialSpan:
                    for rank1paper in yearpublist['rank1']:
                        field = magid2bigcategory[paper2journalid[rank1paper]]
                        if field != '综合性期刊':
                            initialSet.add(field)
                            corepaperlist.append(rank1paper)
                    for rankLastpaper in yearpublist['rankLast']:
                        field = magid2bigcategory[paper2journalid[rankLastpaper]]
                        if field != '综合性期刊':
                            initialSet.add(field)
                            corepaperlist.append(rankLastpaper)
                # 非处女作之年，看是否符合"转向标准"
                else:
                    if Targetfield not in initialSet:
                        return '初始学科非'+Targetfield
                    # 下面可以保证是我们指定的Targetfield
                    for rank1paper in yearpublist['rank1']:
                        # 对这个 rank1paper 默认是转向文献，直到有一个熟悉的学科出现，才不是。
                        flag = 1
                        field = magid2bigcategory[paper2journalid[rank1paper]]
                        # 突破的条件：这本期刊所有的学科都是新的（nonBreakthroughJournal里的不算）；
                        # 如果领域是出现过的，或者是跨学科领域，那么认为不是突破性学科。
                        if (field in initialSet) or (field == '综合性期刊') :
                            flag = 0
                            corepaperlist.append(rank1paper)
                        # 如果全部过了一遍，没有老东西出现就承认是转向了，这一年就是转向年；否则继续去看下一年的文章是否是转向的
                        if flag == 1:
                            # 第n次尝试后终于跨界的标识（0表示是从一而终）；核心作品id list（时间顺序）；跨界作品id
                            return yearid, corepaperlist, rank1paper
                    for rankLastpaper in yearpublist['rankLast']:
                        flag = 1
                        field = magid2bigcategory[paper2journalid[rankLastpaper]]
                        if (field in initialSet) or (field == '综合性期刊') :
                            flag = 0
                            corepaperlist.append(rankLastpaper)
                        if flag == 1:
                            return yearid, corepaperlist, rankLastpaper
    
    
    if Targetfield not in initialSet:
        return '初始学科非'+Targetfield
    # 如果所有的文章都看完了，还没有转向，说明这个人一辈子都没有突破自己第一年的情况；输出0，即没有转向过。
    return 0, corepaperlist

In [18]:
# START_YAER = 1801
# END_YAER = 2021
# cutyearCount = 2
# notmoveAuthorIDandPapers = {}
# moveAuthorIDandPapersandBreak = {}
# import jsonlines
# with jsonlines.open('../../DataCrossBoundaryPerspective_InterdisciplinaryResearch/activeAuthorSeqForCAS.jsonl', mode='r') as reader:
#     for lines in tqdm(reader):
#         for authorid, publist in lines.items():
#             Startyear = StartYearpublist(publist, 'both')
#             TrueSpan = TrueSpanYearpublist(publist, 'both')
#             if ((START_YAER-1) < Startyear < (END_YAER+1)) and ( (cutyearCount-1) < TrueSpan ):
#                 temp = BreakthroughForCAS_IDRidentify(publist,cutYear=100, initialSpan=1,TypeStr='both',Targetfield='管理学')
#                 # 如果不是targetfield，不管
#                 if type(temp) == type('1'):
#                     continue
#                 elif temp[0] == 0:
#                     # 没有跳转，仅记录核心集
#                     notmoveAuthorIDandPapers[authorid] = temp[1]
#                 #成功跳转了的
#                 else:
#                     # 跳转了，核心集，跳转文献
#                     moveAuthorIDandPapersandBreak[authorid]= [temp[1],temp[2]]

In [19]:
# len(notmoveAuthorIDandPapers)

In [20]:
#跨学科性数值转换为对应年份、对应学科的百分位数 value2percentile
def IDRvalue2percentile(paper):
    year = paperYear[paper]
    field = index_map[magid2bigcategory[paper2journalid[paper]]]
    return sum(year_field_iddegree_list[year][field] < paper2idDegree[paper]) /len(year_field_iddegree_list[year][field])

In [21]:
# 突破文献在新学科的百分位数，原学科发文的百分位数, 突破文献在原学科发文的百分位数
# breakPaper2initialPaper = {}
# for k,v in tqdm(moveAuthorIDandPapersandBreak.items()):
#     # v[1]是跳转文献
#     sumPaperIDR = 0
#     for paper in v[0]:
#         sumPaperIDR += IDRvalue2percentile(paper)
#     breakPaper2initialPaper[k] = [IDRvalue2percentile(v[1]),sumPaperIDR/len(v[0])]

In [22]:
# # 增加跳转学科，在管理学中相对百分位数
# for k,v in tqdm(moveAuthorIDandPapersandBreak.items()):
#     # v[1]是跳转文献
#     breakPaper2initialPaper[k].append(sum(year_field_iddegree_list[paperYear[v[1]]][index_map['管理学']] < paper2idDegree[v[1]]) /len(year_field_iddegree_list[paperYear[v[1]]][index_map['管理学']]))

In [23]:
# 把这个东西绘制个箱线图，做个对比；希望说明转向的这个，跨学科程度高很多。
# pk.dump(breakPaper2initialPaper,open('../../DataCrossBoundaryPerspective_InterdisciplinaryResearch/breakPaper2initialPaper.pk', 'wb'))

In [24]:
# 从这中间抽取所有可能的管理学领域研究者及其发文序列
START_YAER = 1801
END_YAER = 2021
cutyearCount = 2
author2publist = {}
import jsonlines
with jsonlines.open('../../DataCrossBoundaryPerspective_InterdisciplinaryResearch/activeAuthorSeqForCAS.jsonl', mode='r') as reader:
    for lines in tqdm(reader):
        for authorid, publist in lines.items():
            Startyear = StartYearpublist(publist, 'both')
            TrueSpan = TrueSpanYearpublist(publist, 'both')
            # 这里实际上对时间没有任何限制，只是对其是否有足够的生涯长度（两年核心发文者经历）进行了识别。
            if ((START_YAER-1) < Startyear < (END_YAER+1)) and ( (cutyearCount-1) < TrueSpan ):
                temp = BreakthroughForCAS_IDRidentify(publist,cutYear=100, initialSpan=1,TypeStr='both',Targetfield='管理学')
                # 如果不是targetfield，不管
                if type(temp) == type('1'):
                    continue
                else:
                    author2publist[authorid]=publist

5866212it [01:45, 55791.66it/s] 


In [25]:
# 这里当然是日历年
author_startyear_endyear = {}
for authorid, publist in author2publist.items():
    author_startyear_endyear[authorid] = [int(sorted(publist.items())[0][0]),int(sorted(publist.items())[-1][0])]

In [26]:

author_startyear_endyear['2122573570']

[2000, 2021]

In [27]:
def chooseSampleTandTime(leftTime,rightTime,author2publist):
    authorlist = []
    for authorid, publist in author2publist.items():
        Startyear = StartYearpublist(publist, 'both')
        # 时间窗口内必须可观察到其初始学科，基本符合
        if Startyear < leftTime or Startyear>rightTime:
            continue
        if author_startyear_endyear[authorid][0]<= leftTime and author_startyear_endyear[authorid][1]>= rightTime:
            authorlist.append(authorid)
    return authorlist

In [28]:
# for lefttime in tqdm(range(2000,2012)):
#     for righttime in range(1990,2016):
#         if righttime-lefttime<5:
#             continue
#         if len(chooseSampleTandTime(lefttime,righttime,author2publist))< 1000:
#             continue
#         else:
#             print(lefttime, righttime, len(chooseSampleTandTime(lefttime,righttime,author2publist)))

生涯开始年份早于2005，结束年份晚于2015。

In [29]:
cited2citing = pk.load(open('../../DataCrossBoundaryPerspective_InterdisciplinaryResearch/cited2citing.pk', 'rb'))

In [30]:
# did author id
authorid = chooseSampleTandTime(2005,2015,author2publist)

In [31]:
# pk.dump(authorid,open('../../DataCrossBoundaryPerspective_InterdisciplinaryResearch/authorid.pk', 'wb'))
# authorid = pk.load(open('../../DataCrossBoundaryPerspective_InterdisciplinaryResearch/authorid.pk', 'rb'))

In [32]:
# # 每个author是否得到了treat
# author_istreat = {}
# author_treatyear = {}
# for author, publist in author2publist.items():
#     # 是我们需要的研究者
#     if author in authorid:
#         temp = BreakthroughForCAS_IDRidentify(publist,cutYear=100, initialSpan=1,TypeStr='both',Targetfield='管理学')
#                 # 如果不是targetfield，不管
#         if type(temp) == type('1'):
#             print('wrong?')
#             continue
#         elif temp[0] == 0:
#                     # 没有跳转，仅记录核心集
#             # notmoveAuthorIDandPapers[authorid] = temp[1]
#             author_istreat[author] = 0
#             author_treatyear[author] = 0
#                 #成功跳转了的
#         else:
#                     # 跳转了，核心集，跳转文献
#             # moveAuthorIDandPapersandBreak[authorid]= [temp[1],temp[2]]
#             if paperYear[temp[2]] >2015:
#                 author_istreat[author] = 0
#                 author_treatyear[author] = 0
#             else:
#                 author_istreat[author] = 1
#                 author_treatyear[author] = paperYear[temp[2]]

In [33]:
# author_istreat['2122573570']

In [34]:
# pk.dump(author_istreat,open('../../DataCrossBoundaryPerspective_InterdisciplinaryResearch/author_istreat.pk', 'wb'))
# pk.dump(author_treatyear,open('../../DataCrossBoundaryPerspective_InterdisciplinaryResearch/author_treatyear.pk', 'wb'))

In [35]:
def paper5yearcitationSumNum(paperset,year):
    '''
    需要一个文献集合
    '''
    if len(paperset) == 0:
        return 0
    citation_count = 0
    for paper in paperset:
        try:
            citation_list = cited2citing[paper]
        except:
            continue
        for citation in citation_list:
            try:
                # 差距小于6年
                if (paperYear[citation] - year) <6:
                    citation_count += 1
            except:
                continue
    return citation_count
def paperCollaboratorsSumNum(paperset):
    '''
    需要一个文献集合
    '''
    if len(paperset) == 0:
        return 0
    collaborator_count = 0
    for paper in paperset:
        try:
            collaborator_count += paperAuthorNum[paper]
        except:
            continue
    return collaborator_count

核心因变量

In [36]:
# 每年的，总发文量、核心发文量
author_year_allpubNum = {}
author_year_corepubNum = {}
# 该年发表的期刊的质量（被引量）
author_year_allpuball5yearcitationNum = {}
author_year_corepuball5yearcitationNum = {}
# 总被引量/总篇数即为平均被引量
author_year_allpubAVG5yearcitationNum = {}
author_year_corepubAVG5yearcitationNum = {}
# 团队数量规模
author_year_allpuballCollaboratorsNum = {}
author_year_corepuballCollaboratorsNum = {}
# 除以总篇数即为平均规模
author_year_allpubAVGCollaboratorsNum = {}
author_year_corepubAVGCollaboratorsNum = {}

StartYear=2005
EndYear=2015
for author, publist in author2publist.items():
    # yearPublist  {2023:{'rank1':list(), 'rankLast':list(), 'others':list() }, ...}
    if author in authorid:
        for year, yearPublist in publist.items():
            year = int(year)
            if StartYear-1< year < EndYear+1:
                # 总发文量
                if author not in author_year_allpubNum:
                    author_year_allpubNum[author]={year:len(yearPublist['rank1'])+len(yearPublist['rankLast'])+len(yearPublist['others'])}
                else:
                    author_year_allpubNum[author][year] = len(yearPublist['rank1'])+len(yearPublist['rankLast'])+len(yearPublist['others'])
                # 核心发文量
                if author not in author_year_corepubNum:
                    author_year_corepubNum[author]={year:len(yearPublist['rank1'])+len(yearPublist['rankLast'])}
                else:
                    author_year_corepubNum[author][year] = len(yearPublist['rank1'])+len(yearPublist['rankLast'])
                # 总被引量
                if author not in author_year_allpuball5yearcitationNum:
                    author_year_allpuball5yearcitationNum[author]={year:paper5yearcitationSumNum(yearPublist['rank1']+yearPublist['others']+yearPublist['rankLast'],year)}
                else:
                    author_year_allpuball5yearcitationNum[author][year] = paper5yearcitationSumNum(yearPublist['rank1']+yearPublist['others']+yearPublist['rankLast'],year)
                # 核心文献被引量
                if author not in author_year_corepuball5yearcitationNum:
                    author_year_corepuball5yearcitationNum[author]={year:paper5yearcitationSumNum(yearPublist['rank1']+yearPublist['rankLast'],year)}
                else:
                    author_year_corepuball5yearcitationNum[author][year] = paper5yearcitationSumNum(yearPublist['rank1']+yearPublist['rankLast'],year)
                # 团队规模
                if author not in author_year_allpuballCollaboratorsNum:
                    author_year_allpuballCollaboratorsNum[author]={year:paperCollaboratorsSumNum(yearPublist['rank1']+yearPublist['others']+yearPublist['rankLast'])}
                else:
                    author_year_allpuballCollaboratorsNum[author][year] = paperCollaboratorsSumNum(yearPublist['rank1']+yearPublist['others']+yearPublist['rankLast'])
                # 核心文献团队规模
                if author not in author_year_corepuballCollaboratorsNum:
                    author_year_corepuballCollaboratorsNum[author]={year:paperCollaboratorsSumNum(yearPublist['rank1']+yearPublist['rankLast'])}
                else:
                    author_year_corepuballCollaboratorsNum[author][year] = paperCollaboratorsSumNum(yearPublist['rank1']+yearPublist['rankLast'])

In [ ]:
# 每年的，总发文量、核心发文量
# author_year_allpubNum = {}
# author_year_corepubNum = {}
# 该年发表的期刊的质量（被引量）
# author_year_allpuball5yearcitationNum = {}
# author_year_corepuball5yearcitationNum = {}
# 总被引量/总篇数即为平均被引量
# author_year_allpubAVG5yearcitationNum = {}
# author_year_corepubAVG5yearcitationNum = {}
# 团队数量规模
# author_year_allpuballCollaboratorsNum = {}
# author_year_corepuballCollaboratorsNum = {}
# 除以总篇数即为平均规模
# author_year_allpubAVGCollaboratorsNum = {}
# author_year_corepubAVGCollaboratorsNum = {}
pk.dump(author_year_allpubNum,open('../../DataCrossBoundaryPerspective_InterdisciplinaryResearch/author_year_allpubNum.pk', 'wb'))
pk.dump(author_year_corepubNum,open('../../DataCrossBoundaryPerspective_InterdisciplinaryResearch/author_year_corepubNum.pk', 'wb'))
pk.dump(author_year_allpuball5yearcitationNum,open('../../DataCrossBoundaryPerspective_InterdisciplinaryResearch/author_year_allpuball5yearcitationNum.pk', 'wb'))
pk.dump(author_year_corepuball5yearcitationNum,open('../../DataCrossBoundaryPerspective_InterdisciplinaryResearch/author_year_corepuball5yearcitationNum.pk', 'wb'))
pk.dump(author_year_allpuballCollaboratorsNum,open('../../DataCrossBoundaryPerspective_InterdisciplinaryResearch/author_year_allpuballCollaboratorsNum.pk', 'wb'))
pk.dump(author_year_corepuballCollaboratorsNum,open('../../DataCrossBoundaryPerspective_InterdisciplinaryResearch/author_year_corepuballCollaboratorsNum.pk', 'wb'))

核心协变量

In [ ]:
# 作者生涯长度!!!!!
author_year_careerLength = {}
# 作者当年跨学科程度的百分位数（平均数）!!!!!
author_year_IDRpercentile ={}
# 滞后一期的累计被引量
author_year_cumuCitationNumLagged1 = {}
# 滞后一期的累计发文数
author_year_cumuPaperNumLagged1 = {}
# 滞后一期的累计合作者人次
author_year_cumuCollaboratorNumLagged1 = {}
# 总参考文献数量!!!!
author_year_totalreferences = {}
#做一个除法可以得到平均参考文献数量
author_year_AVGreferences = {}

In [ ]:
def paperavgIDR(paperset,year):
    IDRsum = 0
    paperCount = 0
    for paper in paperset:
        try:
            IDRsum += IDRvalue2percentile(paper)
            paperCount += 1
        except:
            continue
    try:
        return IDRsum/paperCount
    except:
        return 0

In [ ]:
def refsum(paperset):
    countSum = 0
    for paper in paperset:
        try:
            countSum += referenceCounter[paper]
        except:
            continue
    return countSum

In [ ]:
StartYear=2005
EndYear=2015
for author, publist in author2publist.items():
    # yearPublist  {2023:{'rank1':list(), 'rankLast':list(), 'others':list() }, ...}
    if author in authorid:
        # 职业生涯寿命
        # authorstartyear = int(sorted(publist.items())[0][0])
        # for i in range(StartYear,EndYear+1):
        #     if author not in author_year_careerLength:
        #         author_year_careerLength[author]={i:i-authorstartyear}
        #     else:
        #         author_year_careerLength[author][i] = i-authorstartyear
        # 作者当年跨学科程度的百分位数（平均数）
        for year, yearPublist in publist.items():
            year = int(year)
            if StartYear-1< year < EndYear+1:
                if author not in author_year_IDRpercentile:
                    author_year_IDRpercentile[author]={year:paperavgIDR((yearPublist['rank1']+yearPublist['others']+yearPublist['rankLast']),year)}
                else:
                    author_year_IDRpercentile[author][year] = paperavgIDR((yearPublist['rank1']+yearPublist['others']+yearPublist['rankLast']),year)
                if author not in author_year_totalreferences:
                    author_year_totalreferences[author] = {year:refsum(yearPublist['rank1']+yearPublist['others']+yearPublist['rankLast'])}
                else:
                    author_year_totalreferences[author][year] = refsum(yearPublist['rank1']+yearPublist['others']+yearPublist['rankLast'])

In [ ]:
# author_year_totalreferences
# author_year_IDRpercentile
# author_year_careerLength
pk.dump(author_year_totalreferences,open('../../DataCrossBoundaryPerspective_InterdisciplinaryResearch/author_year_totalreferences.pk', 'wb'))
pk.dump(author_year_IDRpercentile,open('../../DataCrossBoundaryPerspective_InterdisciplinaryResearch/author_year_IDRpercentile.pk', 'wb'))
# pk.dump(author_year_careerLength,open('../../DataCrossBoundaryPerspective_InterdisciplinaryResearch/author_year_careerLength.pk', 'wb'))